In [ ]:
import pandas as pd
import numpy as np
import emoji

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/ETICA/dataset_newtral_ucm.csv')

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 41.5 MB/s 
     |████████████████████████████████| 61 kB 498 kB/s 
     |████████████████████████████████| 596 kB 70.4 MB/s 
     |████████████████████████████████| 895 kB 59.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers
from transformers import pipeline
pipe = pipeline("text-classification", model="Newtral/xlm-r-finetuned-toxic-political-tweets-es")

In [ ]:
pip install emoji

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/ETICA/dataset_newtral_ucm.csv')

In [ ]:
def comprobar_tweet(tweet : str):
  words = tweet.split()
  hayEmoji = False
  listEmojis = str()
  for i in words:
    if emoji.is_emoji(i):
      hayEmoji = True
      listEmojis = listEmojis + " " + i

  return hayEmoji, listEmojis


In [ ]:
column_names = ["twitter_id", "original_text", "slug", "party_slug", "emojis", "toxicity", "toxicity_without_emojis", "sev_toxicity", "sev_toxicity_without_emojis"]
df_wordmod = pd.DataFrame(columns = column_names)
df_wordmod

,twitter_id,original_text,slug,party_slug,emojis,toxicity,toxicity_without_emojis,sev_toxicity,sev_toxicity_without_emojis


In [ ]:
tweets_with_emojis_list = list()


In [ ]:
for idx, row in df.iterrows():
  tweet = row.text
  aux = comprobar_tweet(tweet)
  if aux[0]:
    text_without_emoji = emoji.replace_emoji(row.text, replace="")
    toxic_scores = pipe(text_without_emoji, return_all_scores=True)
    new_row = {'twitter_id':row.twitter_id, 'original_text':row.text, 'slug':row.slug, 'party_slug':row.party_slug, 'emojis': aux[1], 'toxicity':row.toxicity,'toxicity_without_emojis': toxic_scores[0][0]['score'], 'sev_toxicity':row.sev_toxicity, 'sev_toxicity_without_emojis':toxic_scores[0][1]['score']}
    tweets_with_emojis_list.append(new_row)
    df_wordmod = df_wordmod.append(new_row, ignore_index = True) 

In [ ]:
df_wordmod = df_wordmod.drop_duplicates()
df_wordmod = df_wordmod.sort_values('toxicity',ascending=False)

In [ ]:
df_wordmod 

,twitter_id,original_text,slug,party_slug,emojis,toxicity,toxicity_without_emojis,sev_toxicity,sev_toxicity_without_emojis
2923,1403119500384522244,"Ekain Perrino tiene 23 años. \n\nHace días, 13...",gerardo-pisarello-prados,catalunya-en-comu,🌈,0.946050,0.530700,0.976543,0.004655
3055,1412152021155778561,🌈 Los que hoy dicen que matar a un chico de 24...,joan-mena-arca,catalunya-en-comu,🌈,0.881831,0.878469,0.941921,0.006227
2554,1383460145892446216,"😭 Cada vez que pienso que esta señora, señoro ...",jose-angel-alonso-perez,partido-popular,😭,0.867858,0.996455,0.657210,0.788988
2537,1382341705182105610,"“Cerdos endogámicos y pedantes, inútiles, enfe...",pablo-iglesias-turrion,NaN,👇🏻,0.855327,0.993466,0.809204,0.997177
2410,1374444794202615809,Lo que han hecho PSOE y Podemos en Mallorca re...,antonio-pablo-gonzalez-terol,partido-popular,🇪🇸,0.846022,0.995966,0.347208,0.991041
...,...,...,...,...,...,...,...,...,...
1962,1450521151982157828,"Los jóvenes migrantes suelen ser noticia, solo...",jaume-asens-llodra,catalunya-en-comu,👇,NaN,NaN,0.072107,0.002994
1963,1448969875335094274,¡Nos vemos en Valencia! 🚅\n#40CongresoPSOE #Av...,arnau-ramirez-carner,psoe,🚅,NaN,NaN,0.002805,0.003178
1964,1451258690112217094,📻 Mañana participo en la tertulia de las Corte...,ana-maria-zurita-exposito,partido-popular,📻,NaN,NaN,0.025544,0.002739
1965,1449054141918203904,En el #40CongresoPSOE escuchando a Félix Bolañ...,NaN,NaN,🌹,NaN,NaN,0.012849,0.002564


In [ ]:
column_names_emojis = ["emoji", "count", "total_toxicity", "mean", "most_toxic", "most_toxic_text"]
df_wordmod_emojis = pd.DataFrame(columns = column_names_emojis)
df_wordmod_emojis

,emoji,count,total_toxicity,mean,most_toxic,most_toxic_text


In [ ]:
dict_emojis = dict()

In [ ]:
for i in tweets_with_emojis_list:
  aux = i['emoji'].split()
  for j in aux:
    if aux not in dict_emojis:
      dict_emojis.update(aux: 1)
    else:
      dict_emojis.update(aux: dict_emojis[aux] + 1)
